In [8]:
pip install reportlab pillow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
from tkinter import PhotoImage
import mysql.connector
from datetime import date
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from PIL import Image, ImageDraw, ImageFont

# DB connection
con = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='payment_system'
)
cur = con.cursor()

# Main Application
class PaymentApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Employee Payment Management System")
        self.root.geometry("950x500")
        self.root.configure(bg="#fdf6e3")  # pastel beige

        self.entries = {}

        tk.Label(root, text="Employee Payment Management System", font=("Arial", 18, "bold"), bg="#fdf6e3", fg="#6a5acd").pack(pady=10)

        frame = tk.Frame(root, bg="#e0f7fa", bd=2, relief="ridge")
        frame.place(x=20, y=60, width=900, height=300)

        fields = [
            ("Full Name", 0, 0),
            ("Company/Agency", 1, 0),
            ("Hours Worked", 2, 0),
            ("Tax", 3, 0),
            ("GrossPay", 4, 0),
            ("Home Address", 0, 2),
            ("Phone Number", 1, 2),
            ("Hourly Rate", 2, 2),
            ("OverTime", 3, 2),
            ("Net Pay", 4, 2)
        ]

        for label, r, c in fields:
            tk.Label(frame, text=label, bg="#e0f7fa", font=("Arial", 10)).grid(row=r, column=c, padx=10, pady=5, sticky="e")
            ent = tk.Entry(frame, width=25)
            ent.grid(row=r, column=c + 1, padx=5, pady=5)
            self.entries[label] = ent

        # Date Label
        today = date.today().strftime("%d/%m/%Y")
        tk.Label(root, text=today, font=("Arial", 10, "bold"), bg="#c8e6c9").place(x=800, y=30)

        # Buttons
        btn_frame = tk.Frame(root, bg="#fdf6e3")
        btn_frame.place(x=100, y=370)

        tk.Button(btn_frame, text="Weekly Salary", width=20, bg="#b2dfdb", command=self.calculate_salary).grid(row=0, column=0, padx=10)
        tk.Button(btn_frame, text="Reset", width=20, bg="#ffe0b2", command=self.reset_fields).grid(row=0, column=1, padx=10)
        tk.Button(btn_frame, text="View Payslip", width=20, bg="#d1c4e9", command=self.save_to_db).grid(row=0, column=2, padx=10)
        tk.Button(btn_frame, text="Save Payslip", width=20, bg="#b39ddb", command=self.save_payslip).grid(row=0, column=3, padx=10)
        tk.Button(btn_frame, text="Exit System", width=20, bg="#ef9a9a", command=root.quit).grid(row=0, column=4, padx=10)

    def calculate_salary(self):
        try:
            hours = float(self.entries["Hours Worked"].get())
            rate = float(self.entries["Hourly Rate"].get())
            overtime = float(self.entries["OverTime"].get())

            gross = (hours * rate) + (overtime * rate * 1.5)
            tax = gross * 0.1
            net = gross - tax

            self.entries["Tax"].delete(0, tk.END)
            self.entries["GrossPay"].delete(0, tk.END)
            self.entries["Net Pay"].delete(0, tk.END)

            self.entries["Tax"].insert(0, f"{tax:.2f}")
            self.entries["GrossPay"].insert(0, f"{gross:.2f}")
            self.entries["Net Pay"].insert(0, f"{net:.2f}")
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numeric values.")

    def reset_fields(self):
        for entry in self.entries.values():
            entry.delete(0, tk.END)

    def save_to_db(self):
        try:
            data = (
                self.entries["Full Name"].get(),
                self.entries["Company/Agency"].get(),
                self.entries["Home Address"].get(),
                self.entries["Phone Number"].get(),
                float(self.entries["Hours Worked"].get()),
                float(self.entries["Hourly Rate"].get()),
                float(self.entries["OverTime"].get()),
                float(self.entries["Tax"].get()),
                float(self.entries["GrossPay"].get()),
                float(self.entries["Net Pay"].get())
            )
            query = '''
                INSERT INTO employee_payments 
                (full_name, company, address, phone, hours_worked, hourly_rate, overtime, tax, gross_pay, net_pay)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            '''
            cur.execute(query, data)
            con.commit()
            messagebox.showinfo("Success", "Payslip saved to database.")
        except Exception as e:
            messagebox.showerror("Error", f"Could not save to database: {e}")

    def save_payslip(self):
        payslip_data = {
            "Full Name": self.entries["Full Name"].get(),
            "Company/Agency": self.entries["Company/Agency"].get(),
            "Hours Worked": self.entries["Hours Worked"].get(),
            "Hourly Rate": self.entries["Hourly Rate"].get(),
            "OverTime": self.entries["OverTime"].get(),
            "GrossPay": self.entries["GrossPay"].get(),
            "Tax": self.entries["Tax"].get(),
            "Net Pay": self.entries["Net Pay"].get()
        }

        # Ask user for file format
        file_format = simpledialog.askstring("File Format", "Enter file format (pdf/jpeg):").lower()
        if file_format == "pdf":
            self.save_pdf(payslip_data)
        elif file_format == "jpeg":
            self.save_jpeg(payslip_data)
        else:
            messagebox.showerror("Invalid Format", "Only pdf and jpeg formats are supported.")

    def save_pdf(self, data):
        file_name = f"{data['Full Name']}_payslip.pdf"
        c = canvas.Canvas(file_name, pagesize=letter)
        c.drawString(100, 750, f"Employee Payment Payslip")
        c.drawString(100, 730, f"Full Name: {data['Full Name']}")
        c.drawString(100, 710, f"Company: {data['Company/Agency']}")
        c.drawString(100, 690, f"Hours Worked: {data['Hours Worked']}")
        c.drawString(100, 670, f"Hourly Rate: {data['Hourly Rate']}")
        c.drawString(100, 650, f"OverTime: {data['OverTime']}")
        c.drawString(100, 630, f"GrossPay: {data['GrossPay']}")
        c.drawString(100, 610, f"Tax: {data['Tax']}")
        c.drawString(100, 590, f"Net Pay: {data['Net Pay']}")
        c.save()

        # Show Success Graphic
        self.show_success_popup(file_name)

    def save_jpeg(self, data):
        file_name = f"{data['Full Name']}_payslip.jpg"
        img = Image.new('RGB', (600, 400), color='white')
        draw = ImageDraw.Draw(img)
        font = ImageFont.load_default()
        
        y_position = 10
        for key, value in data.items():
            draw.text((10, y_position), f"{key}: {value}", font=font, fill='black')
            y_position += 20

        img.save(file_name)

        # Show Success Graphic
        self.show_success_popup(file_name)

    def show_success_popup(self, file_name):
        success_img = PhotoImage(file="success.png")  # Assuming you have a "success.png" image in the working directory
        success_popup = tk.Toplevel(self.root)
        success_popup.title("Success")
        success_popup.geometry("300x300")
        
        label = tk.Label(success_popup, text=f"Payslip saved as: {file_name}", font=("Arial", 12), fg="green")
        label.pack(pady=20)
        
        img_label = tk.Label(success_popup, image=success_img)
        img_label.image = success_img
        img_label.pack(pady=20)

        ok_button = tk.Button(success_popup, text="OK", command=success_popup.destroy)
        ok_button.pack(pady=10)

root = tk.Tk()
app = PaymentApp(root)
root.mainloop()
